In [5]:
import autogen
from autogen import AssistantAgent, UserProxyAgent

In [4]:
config_list = [
    {
        "model": "",  
        "api_key": '',
        "base_url": "",
        "api_type": "", 
        "api_version": "", 
 }
]

llm_config={
    "timeout": 600,
    "cache_seed": 42,
    "config_list": config_list,
}

In [6]:
from duckduckgo_search import DDGS

def search_internet(query, max_results=1):
    """
    Search Skill
    Search Tool
    Search Internet Tool
    Search Internet Skill
    duckduckgo_search tool
    Performs a search on DuckDuckGo using the duckduckgo_search package.

    :param query: str, the search query.
    :param max_results: int, maximum number of results to return.
    :return: list, search results.
    """
    with DDGS() as ddgs:
        return [r for r in ddgs.text(query, max_results=max_results)]


# Example usage:
# results = search_internet('example query')
# print(results)

[{'title': '20 Basic SQL Query Examples for Beginners | LearnSQL.com', 'href': 'https://learnsql.com/blog/basic-sql-query-examples/', 'body': 'Learn how to write SQL queries to retrieve data from a database using 20 examples. The examples cover selecting columns, filtering, joining, aggregating, and more.'}]


In [ ]:
# create an AssistantAgent instance named "assistant" with the LLM configuration.
assistant = AssistantAgent(name="assistant", llm_config={"config_list": config_list})


In [11]:
all_functions=[
    {
        "name": "search_internet",
        "description": "search internet, Performs a search on DuckDuckGo using the duckduckgo_search package.",
        "parameters": {
            "type": "object",
            "properties": {
                "query": {
                    "type":"string",
                    "description": "the search query to look up in internet"   
                },
                "max_results": {
                    "type": "integer",
                    "description": "number of web pages to return as result"
                }
            },
            "required": ["query"]
        }
        }
    ]


In [24]:
llm_config_user_proxy = {
    "functions":all_functions,
    "config_list": config_list,
    "seed": 100,
    "temperature": 0.7
}

# The llm_config_no_tools remains the same, excluding the 'functions' key.
llm_config_no_tools = {k: v for k, v in llm_config_user_proxy.items() if k != 'functions'}

In [22]:
TRAVEL_AGENT_PROMPT = '''You are a helpful assistant that can suggest a travel itinerary for a user. \
    You are the primary coordinator who will receive suggestions or advice from other agents \
    (local_assistant, language_assistant). You must ensure that the finally plan integrates the \
    suggestions from other agents or team members. \n
Expand this guide into a a full 7-day travel itinerary with detailed per-day plans, \
including weather forecasts, places to eat, packing suggestions, and a budget breakdown. \
You MUST suggest actual places to visit, actual hotels to stay and actual restaurants to go to. \
This itinerary should cover all aspects of the trip, from arrival to departure, \
integrating the city guide information with practical travel logistics. \
Your final answer MUST be a complete expanded travel plan, formatted as markdown, \
encompassing a daily schedule, anticipated weather conditions, recommended clothing and items to pack, \
and a detailed budget, ensuring THE BEST TRIP EVER, Be specific and give it a reason why you picked \
# up each place, what make them special!

YOUR FINAL RESPONSE MUST BE THE COMPLETE PLAN that ends with the word TERMINATE. '''

CITY_SELECTION_PROMPT = '''Analyse and select the best city for the trip based on specific criteria \
    such as weather patterns, seasonal events, and travel costs. Use the search_internet skill to \
    know about weather patterns, seasonal events, and travel costs. \
    This task involves comparing multiple cities, considering factors like current weather conditions, \
    upcoming cultural or seasonal events, and overall travel expenses. \
    Your final answer must be a detailed report on the chosen city, \
    and everything you found out about it, including the actual flight costs, weather forecast and attractions.'''

LOCAL_AGENT_PROMPT = '''As a local expert on this city you must compile an in-depth guide for \
someone traveling there and wanting to have THE BEST trip ever! Gather information about key attractions, \
local customs, special events, and daily activity recommendations using search_internet skill. \
Find the best spots to go to, the kind of place only a local would know. \
This guide should provide a thorough overview of what the city has to offer, \
including hidden gems, cultural hotspots, must-visit landmarks, weather forecasts, and high level costs. \
The final answer must be a comprehensive city guide, rich in cultural insights and practical tips, \
tailored to enhance the travel experience.'''

GROUP_MANAGER_PROMPT = '''You are a helpful assistant skilled at cordinating \
a group of other assistants to solve a task. '''

In [16]:
travel_agent = autogen.UserProxyAgent(name='travel_agent_manager',
                                    human_input_mode="NEVER",
                                    llm_config=llm_config_user_proxy,
                                    max_consecutive_auto_reply=10,
                                    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
                                    code_execution_config=False,
                                    system_message=TRAVEL_AGENT_PROMPT,
                                      )

city_expert = autogen.UserProxyAgent(name='city_selection_expert',
                                    human_input_mode="NEVER",
                                    llm_config=llm_config_user_proxy,
                                    max_consecutive_auto_reply=10,
                                    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
                                    code_execution_config=False,
                                    system_message=CITY_SELECTION_PROMPT,
                                      )

local_agent = autogen.UserProxyAgent(name='local_agent',
                                    human_input_mode="NEVER",
                                    llm_config=llm_config_user_proxy,
                                    max_consecutive_auto_reply=10,
                                    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
                                    code_execution_config=False,
                                    system_message=LOCAL_AGENT_PROMPT,
                                      )

In [30]:
groupchat = autogen.GroupChat(
    agents=[travel_agent, city_expert, local_agent], 
    messages=[], 
    max_round=10
    )

In [33]:
manager = autogen.GroupChatManager(
    groupchat=groupchat, 
    llm_config=llm_config_no_tools,
    max_consecutive_auto_reply=30,    
    is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
    system_message=GROUP_MANAGER_PROMPT
    )

In [34]:
travel_agent.register_function(
    function_map={
        "search_internet": search_internet,
    }
)


result = manager.initiate_chat(
    manager, 
    message='''create a trip plan for visiting london.
'''
    )

chat_manager (to chat_manager):

create a trip plan for visiting london.


--------------------------------------------------------------------------------
city_selection_expert (to chat_manager):

***** Suggested function call: search_internet *****
Arguments: 
{
  "query": "current weather in London",
  "max_results": 3
}
****************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION search_internet...
travel_agent_manager (to chat_manager):

***** Response from calling function (search_internet) *****
[{'title': 'London, London, United Kingdom Current Weather | AccuWeather', 'href': 'https://www.accuweather.com/en/gb/london/ec4a-2/current-weather/328328', 'body': 'Get the current weather conditions and outlook for London, United Kingdom, including temperature, humidity, wind, pressure, and cloud cover. See alerts for flood and severe weather, and check the hourly and daily forecast f

In [44]:
manager.chat_messages.keys()

dict_keys([<autogen.agentchat.groupchat.GroupChatManager object at 0x7f45b83fef50>, <autogen.agentchat.user_proxy_agent.UserProxyAgent object at 0x7f45d0232c50>, <autogen.agentchat.user_proxy_agent.UserProxyAgent object at 0x7f45d02f0730>, <autogen.agentchat.user_proxy_agent.UserProxyAgent object at 0x7f45d00ba560>, -1])

In [48]:
print(manager.chat_messages[list(manager.chat_messages.keys())[1]][-1]['content'])

# 7-Day Travel Itinerary for London 

## Day 1: Arrival in London
Arrive at London Heathrow Airport and take a cab to the **Kimpton Fitzroy London Hotel**. This hotel is a tried and tested choice among travellers, known for its ornate architecture and excellent service. Unpack, freshen up and get ready to start your London adventure.

#### Places to visit:
- **British Museum**: Explore this iconic museum that houses a vast collection of world art and artifacts. 

#### Places to eat:
- **Davy's**: Try the local favorite, Bangers & Mash, a traditional dish in London.

## Day 2: Exploring London's Landmarks
#### Places to visit:
- **Buckingham Palace**: Witness the changing of the guard, a quintessential London experience.
- **Tower of London**: Visit this historic castle that houses the Crown Jewels. 

#### Places to eat:
- **Gymkhana**: Enjoy a meal at one of London's best restaurants.

## Day 3: Discovering London's Cultural Gems
#### Places to visit:
- **National Gallery**: Admire mas